In [1]:
import pandas as pd

from src.utils import download_images 
#src\utils.py

In [2]:
import os 
os.getcwd()

'c:\\Users\\manas\\OneDrive\\Documents\\GitHub\\amazonml\\__pycache__'

In [3]:
import pandas as pd
df = pd.read_csv('src/sample_test.csv')

In [4]:
df_86 = df.sample(86)
df_86

,index,image_link,group_id,entity_name
54,54,https://m.media-amazon.com/images/I/51y79cwGJF...,200507,depth
55,55,https://m.media-amazon.com/images/I/51y79cwGJF...,200507,height
51,51,https://m.media-amazon.com/images/I/51tEop-EBJ...,124643,wattage
31,31,https://m.media-amazon.com/images/I/51FSlaVlej...,150535,height
85,85,https://m.media-amazon.com/images/I/81aZ2ozp1G...,805279,maximum_weight_recommendation
...,...,...,...,...
5,5,https://m.media-amazon.com/images/I/41ADVPQgZO...,993359,item_weight
78,78,https://m.media-amazon.com/images/I/71fWddA0+y...,318770,wattage
60,60,https://m.media-amazon.com/images/I/615Cjzm6py...,759408,height
84,84,https://m.media-amazon.com/images/I/81PG3ea0MO...,240413,voltage


In [5]:
images_links = df_86['image_link'].to_list()

In [6]:
import cv2
import pytesseract
import numpy as np
import os
import matplotlib.pyplot as plt

# Function to display multiple images in a grid
def display_images(images, title):
    # Adjust the grid size for 86 images (e.g., 18 rows and 5 columns to accommodate)
    fig, axes = plt.subplots(18, 5, figsize=(15, 30))  # 18 rows, 5 columns (90 images capacity)
    fig.suptitle(title, fontsize=16)

    for i, img in enumerate(images):
        if i >= 86:  # Limit to 86 images
            break
        ax = axes[i // 5, i % 5]
        ax.imshow(img, cmap='gray')
        ax.axis('off')
        ax.set_title(f'Image {i+1}')
    
    plt.tight_layout()
    plt.show()

# Initialize an array to store the images
image_paths = [os.path.join('sample_images', filename) for filename in os.listdir('sample_images') if filename.endswith(('.png', '.jpg', '.jpeg'))]

# Read only up to 86 images
original_images = [cv2.imread(path) for path in image_paths[:86] if cv2.imread(path) is not None]



In [7]:
# Grayscale conversion
gray_images = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in original_images]



In [8]:
# Apply Otsu's binarization
binary_images = [cv2.threshold(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1] for img in original_images]

In [9]:
# 4. Noise removal using Median Blur and display
denoised_images = [cv2.medianBlur(binary, 3) for binary in binary_images]

In [10]:
# 5. Dilation and display
kernel = np.ones((1, 1), np.uint8)
dilated_images = [cv2.dilate(denoised, kernel, iterations=1) for denoised in denoised_images]


In [11]:
# 6. Erosion and display
eroded_images = [cv2.erode(dilated, kernel, iterations=1) for dilated in dilated_images]


In [12]:
# 7. Skew Correction (rotation) and display
def correct_skew(image):
    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated
skew_corrected_images = [correct_skew(eroded) for eroded in eroded_images]

In [13]:
# 8. Resize images (increase size for better OCR performance)
resized_images = [cv2.resize(image, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC) for image in skew_corrected_images]

In [14]:
# 9. Invert colors for white text on black background (optional, based on image content)
inverted_images = [cv2.bitwise_not(image) for image in resized_images]

In [15]:
# 10. OCR Processing on each preprocessed image
ocr_results = []
for img in original_images:
    text = pytesseract.image_to_string(img)
    ocr_results.append(text)

In [16]:
# Output OCR results for each image
for idx, text in enumerate(ocr_results):
    print(f"Text from image {idx+1}:\n{text}\n")

Text from image 1:
30mm/1.18in

30mm/1.18in



Text from image 2:


Text from image 3:
4.37 in/ 11.1cm

8.74 in/ 22.2 cm


Text from image 4:


Text from image 5:
RADIUS: 16 inch /406 mm

Neatly fret , C shape back profile .comfortable to hold



Text from image 6:
Pink | Blue
31.5CM



Text from image 7:


Text from image 8:
F26 PARAMETRO PRO

Telaio

Lega di alluminio

Freno

Freni a disco meccanici

Chilometraggio

Pedalata assistita 70-90km

Batteria
36V 10Ah agli ioni di litio

Tempo di carica
6 Ore

Il motore
Trazione posteriore 250W

Peso della bicicletta
23.2kg

Schermo
Display LCD a colori

Smorzamento

Forcella ammortizzata anteriore

Pneumatico
Pneumatici slim 26*1,75

Controllore

Conirollo intelligente a 5 turni

Trasmissione
SHIMANO 7 marce

Velocita
Massimo 25 km/h

Dimensione della bicicletta
173*109*67em



Text from image 9:


Text from image 10:
Fru favour foam gums
Vorecents: Glucose

Teast Civae:A

‘of which Saturates

Carbohydrate
‘of which Sugars
Protein
Salt
Mad

In [17]:
import re
import string

def preprocess_ocr_text(ocr_results):
  
    
    # Remove unwanted characters (e.g. punctuation, special characters)
    ocr_results = re.sub(r'[^a-zA-Z0-9\s]', '', ocr_results)

    # Convert to lowercase
    # ocr_results = ocr_results.lower()

    # Remove extra whitespace
    ocr_results = re.sub(r'\s+', ' ', ocr_results)

    # Remove leading and trailing whitespace
    ocr_results = ocr_results.strip()

    return ocr_results

In [18]:
prep_text = []
for text in ocr_results:
  preprocessed_text = preprocess_ocr_text(text)
  prep_text.append(preprocessed_text)


In [19]:
print(prep_text)

['30mm118in 30mm118in', '', '437 in 111cm 874 in 222 cm', '', 'RADIUS 16 inch 406 mm Neatly fret C shape back profile comfortable to hold', 'Pink Blue 315CM', '', 'F26 PARAMETRO PRO Telaio Lega di alluminio Freno Freni a disco meccanici Chilometraggio Pedalata assistita 7090km Batteria 36V 10Ah agli ioni di litio Tempo di carica 6 Ore Il motore Trazione posteriore 250W Peso della bicicletta 232kg Schermo Display LCD a colori Smorzamento Forcella ammortizzata anteriore Pneumatico Pneumatici slim 26175 Controllore Conirollo intelligente a 5 turni Trasmissione SHIMANO 7 marce Velocita Massimo 25 kmh Dimensione della bicicletta 17310967em', '', 'Fru favour foam gums Vorecents Glucose Teast CivaeA of which Saturates Carbohydrate of which Sugars Protein Salt Made in the E1 Swizzel 120 COUNT Sugar Gelli nt ave Sich ee Ace Regie Pulp Glazing Sneak iri Paprika UU fs Matlow Ltd New Mills Derbyshire SK22 Swizels and Fun Gums are Registered Trade Marks of Matlow Ltd Best Before see back of Store i

In [20]:
for text in ocr_results:
    print(f" Text: {text}")

 Text: 30mm/1.18in

30mm/1.18in


 Text: 
 Text: 4.37 in/ 11.1cm

8.74 in/ 22.2 cm

 Text: 
 Text: RADIUS: 16 inch /406 mm

Neatly fret , C shape back profile .comfortable to hold


 Text: Pink | Blue
31.5CM


 Text: 
 Text: F26 PARAMETRO PRO

Telaio

Lega di alluminio

Freno

Freni a disco meccanici

Chilometraggio

Pedalata assistita 70-90km

Batteria
36V 10Ah agli ioni di litio

Tempo di carica
6 Ore

Il motore
Trazione posteriore 250W

Peso della bicicletta
23.2kg

Schermo
Display LCD a colori

Smorzamento

Forcella ammortizzata anteriore

Pneumatico
Pneumatici slim 26*1,75

Controllore

Conirollo intelligente a 5 turni

Trasmissione
SHIMANO 7 marce

Velocita
Massimo 25 km/h

Dimensione della bicicletta
173*109*67em


 Text: 
 Text: Fru favour foam gums
Vorecents: Glucose

Teast Civae:A

‘of which Saturates

Carbohydrate
‘of which Sugars
Protein
Salt
Made in the E1
Swizzel

120 COUNT

Sugar, Gelli nt ave
Sich, ee Ace Regie
Pulp Glazing

Sneak iri Paprika

UU
fs Matlow Ltd., New Mil

In [21]:
import csv
import spacy

nlp = spacy.load('en_core_web_sm')

# Open a CSV file to write the entity results
with open('ocr_entities.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # Write the header for the CSV file
    writer.writerow(['id','Entity', 'Label'])
    
    # Loop through OCR results and extract entities
    for text in ocr_results:
        doc = nlp(text)
        for entity in doc.ents:
            if entity.label_ == "QUANTITY":
                # Write entity and label to the CSV
                writer.writerow([entity.start,entity.text, entity.label_])

print("CSV file 'ocr_entities.csv' has been created.")


CSV file 'ocr_entities.csv' has been created.


In [22]:
df = pd.read_csv('ocr_entities.csv')

In [23]:
df.head()

,id,Entity,Label
0,2,11.1cm,QUANTITY
1,7,22.2 cm,QUANTITY
2,2,16 inch,QUANTITY
3,53,23.2kg,QUANTITY
4,94,25 km,QUANTITY


In [24]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load('en_core_web_sm')
matcher = Matcher(nlp.vocab)

pattern = [{"TEXT": {"REGEX": r"\d+(\.\d+)?[a-z]+(/\d+(\.\d+)?[a-z]+)?"}}]
matcher.add("QUANTITY_PATTERN", [pattern])

for text in ocr_results:
    doc = nlp(text)
    matches = matcher(doc)
    for match_id, start, end in matches:
        entity = doc[start:end]
        print(f"Entity: {start}, {entity.text}, Label: QUANTITY")

Entity: 0, 30mm/1.18, Label: QUANTITY
Entity: 3, 30mm/1.18, Label: QUANTITY
Entity: 107, 67em, Label: QUANTITY


In [25]:
import csv

# Open a CSV file to write the entity results
with open('ocr_entities.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # Write the header for the CSV file
    writer.writerow(['id','Entity', 'Label'])
    
    # Loop through OCR results and extract entities
    for text in ocr_results:
        doc = nlp(text)
        for entity in doc.ents:
            if entity.label_ == "QUANTITY":
                # Write entity and label to the CSV
                writer.writerow([entity.start,entity.text, entity.label_])

print("CSV file 'ocr_entities.csv' has been created.")


CSV file 'ocr_entities.csv' has been created.


In [26]:
df = pd.read_csv('ocr_entities.csv')

In [27]:
df.head()

,id,Entity,Label
0,2,11.1cm,QUANTITY
1,7,22.2 cm,QUANTITY
2,2,16 inch,QUANTITY
3,53,23.2kg,QUANTITY
4,94,25 km,QUANTITY


In [28]:
for i in range(len(df)):
    print(df.iloc[i]['Entity'])

11.1cm
22.2 cm
16 inch
23.2kg
25 km


In [29]:
te=()
for i in range(len(df)):
    print(df.iloc[i]['Entity'])
    buff=""
    for i in df.iloc[i]['Entity']:
      if i=='/':
        break
      #print(i)
      buff=buff+i
    print(buff)
    te=te+(buff,)
print(te)
df['Entity']=te

11.1cm
11.1cm
22.2 cm
22.2 cm
16 inch
16 inch
23.2kg
23.2kg
25 km
25 km
('11.1cm', '22.2 cm', '16 inch', '23.2kg', '25 km')


In [30]:
df.head()

,id,Entity,Label
0,2,11.1cm,QUANTITY
1,7,22.2 cm,QUANTITY
2,2,16 inch,QUANTITY
3,53,23.2kg,QUANTITY
4,94,25 km,QUANTITY


In [60]:
import csv
import pandas as pd

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('output.csv')

# Get the value of n from the number of rows in the DataFrame
n = len(df)

# Create an empty dictionary to store the data
m = {}

# Iterate over the rows of the DataFrame
for id, row in df.iterrows():
    if pd.notna(row['id']):
        a = int(row['id'])
        b = row['Entity']
        if a not in m:
            m[a] = []
        m[a].append(b)

# Create a new DataFrame to store the results
result_df = pd.DataFrame(columns=['index', 'Entity'])

# Iterate over the dictionary and create new rows for the result DataFrame
# Create a list to store the new rows
new_rows = []

# Iterate over the dictionary and create new rows
for i in range(1, 88):
    if i in m:
        new_rows.append({'index': i, 'Entity': m[i][0]})
    else:
        new_rows.append({'index': i, 'Entity': ''})

# Convert the list of dictionaries to a DataFrame
new_df = pd.DataFrame(new_rows)

# Concatenate the new DataFrame with the original DataFrame
result_df = pd.concat([result_df, new_df], ignore_index=True)

In [64]:
result_df.to_csv('outputx.csv', index=False)


In [73]:
import csv
import pandas as pd
import re

# Mapping of short forms to full forms
unit_mapping = {
    'width': {'cm': 'centimetre', 'ft': 'foot', 'in': 'inch', 'm': 'metre', 'mm': 'millimetre', 'yd': 'yard'},
    'depth': {'cm': 'centimetre', 'ft': 'foot', 'in': 'inch', 'm': 'metre', 'mm': 'millimetre', 'yd': 'yard'},
    'height': {'cm': 'centimetre', 'ft': 'foot', 'in': 'inch', 'm': 'metre', 'mm': 'millimetre', 'yd': 'yard'},
    'item_weight': {'g': 'gram', 'kg': 'kilogram', 'µg': 'microgram', 'mg': 'milligram', 'oz': 'ounce', 'lb': 'pound', 't': 'ton'},
    'maximum_weight_recommendation': {'g': 'gram', 'kg': 'kilogram', 'µg': 'microgram', 'mg': 'milligram', 'oz': 'ounce', 'lb': 'pound', 't': 'ton'},
    'voltage': {'kV': 'kilovolt', 'mV': 'millivolt', 'V': 'volt'},
    'wattage': {'kW': 'kilowatt', 'W': 'watt'},
    'item_volume': {'cl': 'centilitre', 'ft³': 'cubic foot', 'in³': 'cubic inch', 'cup': 'cup', 'dl': 'decilitre', 'fl oz': 'fluid ounce', 'gal': 'gallon', 'imp gal': 'imperial gallon', 'l': 'litre', 'µl': 'microlitre', 'ml': 'millilitre', 'pt': 'pint', 'qt': 'quart'}
}

def convert_units(row):
    if len(row) < 3:
        return row  # Return the row as is if it has fewer than 3 elements
    ud, value, label = row
    for short_form, full_form in unit_mapping.items():
        pattern = r'(\d+)(?:' + re.escape(' ') + re.escape(short_form) + '|' + re.escape(short_form) + ')'
        value = re.sub(pattern, r'\1 ' + full_form, value)
    return [ud, value, label]

with open('output.csv', 'r') as input_file, open('outputx.csv', 'w', newline='') as output_file:
    reader = csv.reader(input_file)
    writer = csv.writer(output_file)
   
    for row in reader:
        # print(row)
        writer.writerow(convert_units(row))

In [76]:
df = pd.read_csv('outputx.csv')

In [77]:
df.head()

,index,Entity
0,1,NaN
1,2,11.1cm
2,3,NaN
3,4,NaN
4,5,NaN


In [69]:
# Print any issues with indices
print("Missing index in file:", missing_indices)
print("Extra index in file:", extra_indices)

NameError: name 'missing_indices' is not defined

In [79]:
df.to_csv('output_cleaned.csv', index=False)

In [45]:
df.head()

,id,Entity,Label
0,2,11.1cm,QUANTITY
1,7,22.2 cm,QUANTITY
2,2,16 inch,QUANTITY
3,53,23.2kg,QUANTITY
4,94,25 km,QUANTITY


In [85]:
result_df = pd.DataFrame(columns=['index', 'prediction'])

In [49]:
print("File cleaned and saved as 'output_cleaned.csv'")

File cleaned and saved as 'output_cleaned.csv'


In [30]:
import csv
import pandas as pd

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('output_cleaned.csv')

# Get the value of n from the number of rows in the DataFrame
n = len(df)

# Create an empty dictionary to store the data
m = {}

# Iterate over the rows of the DataFrame
for index, row in df.iterrows():
    if pd.notna(row['index']):
        a = int(row['index'])
        b = row['prediction']
        if a not in m:
            m[a] = []
        m[a].append(b)

# Create a new DataFrame to store the results
result_df = pd.DataFrame(columns=['id', 'prediction'])



In [31]:
# Create a list to store the new rows
new_rows = []

# Iterate over the dictionary and create new rows
for i in range(0, 88):
    if i in m:
        new_rows.append({'id': i, 'prediction': m[i][0]})
    else:
        new_rows.append({'id': i, 'prediction': ''})

# Convert the list of dictionaries to a DataFrame
new_df = pd.DataFrame(new_rows)

# Concatenate the new DataFrame with the original DataFrame
result_df = pd.concat([result_df, new_df], ignore_index=True)

In [32]:
result_df

,id,prediction
0,0,69 centimetre / 27.2 inchch
1,1,
2,2,4.2 Inches
3,3,
4,4,
...,...,...
83,83,
84,84,
85,85,
86,86,


In [33]:
result_df.to_csv('output1.csv', index=False)

In [34]:
result_df = result_df.rename(columns={'id': 'index'})

In [35]:
result_df.to_csv('output1.csv', index=False)